In [1]:
import requests
import pandas as pd 
import numpy as np
from os import listdir
from os.path import isfile, join
import pathlib
import urllib
import urllib.request
import zipfile
import io

def buildUrl(year, month):
    return "https://s3.amazonaws.com/capitalbikeshare-data/" + str(year) + "{0:0=2d}".format(month) + "-capitalbikeshare-tripdata.zip"

def downloadFile(url):
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    ourPath =  str(pathlib.Path().absolute()) + "\\data\\bikeShare\\monthlyData\\"
    print(ourPath)
    z.extractall(ourPath)

def downloadAll():
    year = 2019
    month = 2
    while month < 13:
        url = buildUrl(year,month)
        downloadFile(url)

        month += 1
    
def makeFrame():
    ourPath = str(pathlib.Path().absolute()) + "\\data\\bikeShare\\monthlyData\\"

    onlyFiles = [f for f in listdir(ourPath) if isfile(join(ourPath, f))]

    dfs = []
    for fileName in onlyFiles:
        fileName = str(ourPath + fileName)
        print(fileName)
        pd.read_csv(fileName, encoding = "utf-8")
        dfs.append(pd.read_csv(fileName))

    # Concatenate all data into one DataFrame
    big_frame = pd.concat(dfs, ignore_index=True)
    return big_frame
    

In [2]:
downloadAll()
big_frame = makeFrame()
bikeShareLocations = pd.read_csv("data/bikeShareLocations.csv")

C:\Users\jagluck\Documents\Github\dc-bike-analysis\data\bikeShare\monthlyData\
C:\Users\jagluck\Documents\Github\dc-bike-analysis\data\bikeShare\monthlyData\
C:\Users\jagluck\Documents\Github\dc-bike-analysis\data\bikeShare\monthlyData\
C:\Users\jagluck\Documents\Github\dc-bike-analysis\data\bikeShare\monthlyData\
C:\Users\jagluck\Documents\Github\dc-bike-analysis\data\bikeShare\monthlyData\
C:\Users\jagluck\Documents\Github\dc-bike-analysis\data\bikeShare\monthlyData\
C:\Users\jagluck\Documents\Github\dc-bike-analysis\data\bikeShare\monthlyData\
C:\Users\jagluck\Documents\Github\dc-bike-analysis\data\bikeShare\monthlyData\
C:\Users\jagluck\Documents\Github\dc-bike-analysis\data\bikeShare\monthlyData\
C:\Users\jagluck\Documents\Github\dc-bike-analysis\data\bikeShare\monthlyData\
C:\Users\jagluck\Documents\Github\dc-bike-analysis\data\bikeShare\monthlyData\
C:\Users\jagluck\Documents\Github\dc-bike-analysis\data\bikeShare\monthlyData\201901-capitalbikeshare-tripdata.csv
C:\Users\jagluck

In [3]:
big_frame.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member
1,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual
2,177,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654,Casual
3,228,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336,Member
4,1300,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004,Member


In [45]:
startLocations = big_frame['Start station'].values
endLocations = big_frame['End station'].values
allLocations = [i.lower().strip() for i in list(np.unique(np.concatenate((startLocations, endLocations), axis=0)))]
locationList = bikeShareLocations['ADDRESS'].apply(lambda x:x.lower().strip())

In [57]:
print(len(allLocations))
print(len(locationList))

651
579


In [55]:
b = 0
for i in locationList:
    if (i in allLocations):
        b = b + 1
print(b)

577


In [7]:
fr = []
for ss, es in zip(big_frame['Start station'], big_frame['End station']):
    fr.append(ss + ' - ' + es)

In [ ]:
# # big_frame['fullRoutes'] = [big_frame['Start station'] + " - " + big_frame['End station']]
# fr = [big_frame['Start station'] + " - " + big_frame['End station']]
# print(fr)
# print(len(fr))
# print(len(big_frame))
# big_frame['fullRoutes'] = fr

In [10]:
print(list(bikeShareLocations))

['OBJECTID', 'ID', 'ADDRESS', 'TERMINAL_NUMBER', 'LATITUDE', 'LONGITUDE', 'INSTALLED', 'LOCKED', 'INSTALL_DATE', 'REMOVAL_DATE', 'TEMPORARY_INSTALL', 'NUMBER_OF_BIKES', 'NUMBER_OF_EMPTY_DOCKS', 'X', 'Y', 'SE_ANNO_CAD_DATA', 'OWNER']


In [11]:
bikeShareLocations.head()

,OBJECTID,ID,ADDRESS,TERMINAL_NUMBER,LATITUDE,LONGITUDE,INSTALLED,LOCKED,INSTALL_DATE,REMOVAL_DATE,TEMPORARY_INSTALL,NUMBER_OF_BIKES,NUMBER_OF_EMPTY_DOCKS,X,Y,SE_ANNO_CAD_DATA,OWNER
0,301560082,77,Kennedy Center,31211,38.897293,-77.055570,YES,NO,NaN,NaN,NO,16,2,395179.460813,136606.270059,NaN,DC
1,301560083,78,Eastern Market Metro / Pennsylvania Ave & 7th ...,31613,38.884000,-76.995397,YES,NO,NaN,NaN,NO,2,17,400399.371566,135129.185818,NaN,DC
2,301560084,79,18th & M St NW,31221,38.905067,-77.041779,YES,NO,NaN,NaN,NO,11,16,396376.185625,137468.608591,NaN,DC
3,301560085,80,Tenleytown / Wisconsin Ave & Albemarle St NW,31303,38.947607,-77.079382,YES,NO,NaN,NaN,NO,1,14,393118.694901,142193.075135,NaN,DC
4,301560086,81,13th & H St NE,31611,38.900022,-76.988422,YES,NO,NaN,NaN,NO,11,14,401004.320153,136907.807348,NaN,DC


In [12]:
big_frame['fullRoutes'] = fr
# big_frame['fullRoutes'].value_counts()